# Exam: a straightforward, minimal solution

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import animation as ani, pyplot as plt
import seaborn as sns

from IPython.display import HTML

plt.style.use('seaborn-darkgrid')

import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [2]:
data_file_path = ''
# reading the training set
data = pd.read_csv(data_file_path+'train_V2.csv')
# reading the scoring set
score = pd.read_csv(data_file_path+'score.csv')

These data sets are like real life data: messy, with mistakes, and unclarities about the exact meaning of features. The golden rule is: whenever you get into trouble, make assumptions until something starts to make sense. Therefore, we also don't give you too much input about the features - you should show that you can figure things out by yourself. 
        
I'm going to give one extremely simplified example solution, just to show you what you could do. It is perfectly fine to stay close to this solution, although some technical improvements should hopefully be obvious after having taken the course. 

My approach is going to be the following: I'm going to assume that I want to predict the profit, that I want to predict who might have a damage incident (so I will make that binary), and for how much that would then be (i.e. the expected damage per incident, for a given client profile). This is obviously just one possible approach, but it might be ok under some assumptions - up to you to be aware of what they are! Again, refinements are certainly possible, and a host of interesting choices can be made as an analist here!

### Preparation of data

You should first focus on some data steps
- coding categorical features we've seen in the course, so I won't do too much here
- real feature engineering: perhaps you can take log transforms to get meaningful scales, or dividing things by number of nights to get more meaningful features,...
- rescaling features
- handling of data errors

Ideally these things are inspired by a nice exploratory analysis, but hey: this is a *minimal* solution!

Finally, a very important consideration is that everything you do has to be redone for the scoring set, exactly as it was done for the training set! To avoid a mess with mean imputations and categorization etc, I will just stitch the two together, and do the changes - this is allowed, as long as you don't use the outcome!

In [3]:
print(data.shape)
print(data.head())
pd.options.display.max_columns = None
data.describe()

(5000, 53)
   income_am  profit_last_am  profit_am  damage_am  damage_inc  crd_lim_rec  \
0      227.0             0.0     3201.0      888.0         6.0      15000.0   
1      268.0            16.0     1682.0        0.0         0.0        750.0   
2      283.0            23.0     1673.0        0.0         0.0        750.0   
3      227.0             0.0     1685.0        0.0         0.0          0.0   
4     4091.0          1028.0     3425.0      785.0         2.0      14000.0   

   credit_use_ic  gluten_ic  lactose_ic  insurance_ic  ...  score2_neg  \
0            0.0        0.0         0.0           0.0  ...         NaN   
1            0.0        0.0         0.0           1.0  ...         NaN   
2            0.0        0.0         0.0           1.0  ...    0.099529   
3            0.0        0.0         0.0           0.0  ...         NaN   
4            0.0        0.0         1.0           0.0  ...         NaN   

   score3_pos  score3_neg  score4_pos  score4_neg  score5_pos  score5

,income_am,profit_last_am,profit_am,damage_am,damage_inc,crd_lim_rec,credit_use_ic,gluten_ic,lactose_ic,insurance_ic,spa_ic,empl_ic,cab_requests,bar_no,sport_ic,neighbor_income,age,marketing_permit,urban_ic,dining_ic,presidential,client_segment,sect_empl,prev_stay,prev_all_in_stay,divorce,fam_adult_size,children_no,tenure_mts,tenure_yrs,company_ic,claims_no,claims_am,nights_booked,shop_am,shop_use,retired,gold_status,score1_pos,score1_neg,score2_pos,score2_neg,score3_pos,score3_neg,score4_pos,score4_neg,score5_pos,score5_neg,outcome_profit,outcome_damage_inc,outcome_damage_amount
count,4947.000000,4947.000000,4947.000000,4954.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4970.000000,4999.000000,4912.000000,4947.000000,4947.000000,4761.000000,4947.000000,4947.000000,4947.000000,4912.000000,4912.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4608.000000,4608.000000,4947.000000,4947.000000,4973.000000,4947.000000,4947.000000,4912.000000,4947.000000,4947.000000,1.225000e+03,1.314000e+03,1.209000e+03,1.304000e+03,1.261000e+03,1.367000e+03,1.223000e+03,1.324000e+03,1.232000e+03,1493.000000,5000.000000,5000.000000,5000.000000
mean,2281.260158,696.057712,3637.900950,145.952967,0.352335,3298.716394,0.041237,0.024661,0.094199,0.390944,0.401811,0.024205,6.051507,5.646250,0.287043,32778.558916,44.901152,0.495452,0.883970,0.049267,0.004275,1.298565,0.213463,0.889832,0.252678,0.102486,1.960986,0.385082,273.111545,22.780165,0.018597,0.218314,121.078826,28.992521,403.019960,0.151873,0.182131,0.034769,4.997356e-01,5.003663e-01,4.985522e-01,4.967340e-01,4.942801e-01,4.985876e-01,4.962065e-01,5.013962e-01,5.009593e-01,5.192953,1967.310930,0.255400,189.970736
std,8365.254507,3051.119275,5726.625669,581.068095,0.889449,4549.646039,0.198858,0.155107,0.292134,0.488011,0.490313,0.153700,3.112104,5.052513,0.452427,6858.671948,16.225094,0.500030,0.320293,0.216447,0.065252,0.800831,0.826006,0.313130,0.434592,0.303317,0.805545,0.832933,152.498416,12.719429,0.135111,0.712408,1783.146726,37.480510,1335.935144,0.358934,0.385991,0.183212,2.879255e-01,2.887168e-01,2.877572e-01,2.897994e-01,2.899165e-01,2.877292e-01,2.886538e-01,2.876226e-01,2.901323e-01,3.159868,1371.061266,0.436129,379.005941
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,-7.871775,10.680000,0.000000,0.000000
25%,229.000000,0.000000,1638.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,2.000000,0.000000,28630.000000,31.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,154.000000,13.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,2.520205e-01,2.510338e-01,2.521282e-01,2.454209e-01,2.405574e-01,2.495061e-01,2.474100e-01,2.506703e-01,2.514905e-01,3.124958,1333.320000,0.000000,0.000000
50%,469.000000,52.000000,1889.000000,0.000000,0.000000,1500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,5.000000,0.000000,31990.000000,45.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,271.000000,23.000000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,4.974162e-01,4.986215e-01,4.987791e-01,4.985832e-01,4.942465e-01,5.016458e-01,4.933486e-01,5.020603e-01,5.029121e-01,5.188006,1721.235000,0.000000,0.000000
75%,1688.000000,810.000000,3165.500000,0.000000,0.000000,5000.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,9.000000,8.000000,1.000000,35924.000000,57.000000,1.000000,1.00

In [4]:
(data.isnull().sum(axis=1))[data.isnull().sum(axis=1) > 30]

90      47
306     48
474     47
517     47
670     47
790     47
996     48
1062    47
1164    46
1261    47
1275    47
1281    46
1338    46
1448    47
1696    47
1744    48
1852    47
2058    47
2096    47
2250    47
2348    48
2672    48
2696    47
2734    47
2917    47
2956    46
3068    48
3198    46
3213    47
3326    46
3377    48
3494    46
3793    47
3926    46
3952    47
3959    47
3990    47
4000    47
4009    46
4138    47
4159    47
4242    47
4392    47
4477    47
4528    46
4545    48
4614    47
4641    48
4669    46
4789    47
4851    47
4946    47
4993    47
dtype: int64

In [5]:
data.columns

Index(['income_am', 'profit_last_am', 'profit_am', 'damage_am', 'damage_inc',
       'crd_lim_rec', 'credit_use_ic', 'gluten_ic', 'lactose_ic',
       'insurance_ic', 'spa_ic', 'empl_ic', 'cab_requests', 'married_cd',
       'bar_no', 'sport_ic', 'neighbor_income', 'age', 'marketing_permit',
       'urban_ic', 'dining_ic', 'presidential', 'client_segment', 'sect_empl',
       'prev_stay', 'prev_all_in_stay', 'divorce', 'fam_adult_size',
       'children_no', 'tenure_mts', 'tenure_yrs', 'company_ic', 'claims_no',
       'claims_am', 'nights_booked', 'gender', 'shop_am', 'shop_use',
       'retired', 'gold_status', 'score1_pos', 'score1_neg', 'score2_pos',
       'score2_neg', 'score3_pos', 'score3_neg', 'score4_pos', 'score4_neg',
       'score5_pos', 'score5_neg', 'outcome_profit', 'outcome_damage_inc',
       'outcome_damage_amount'],
      dtype='object')

In [6]:
print(data.shape)
data_feat = data.drop(['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount'], axis=1)
print(data_feat.shape)

(5000, 53)
(5000, 50)


In [7]:
print(data_feat.shape)
print(score.shape)
datafull = pd.concat([data_feat, score])
print(datafull.shape)

(5000, 50)
(500, 50)
(5500, 50)


Let's think about some potentially categorical features?

In [8]:
print(datafull['client_segment'].value_counts())
print(datafull['sect_empl'].value_counts())
datafull['gender'].value_counts()

1.0    3712
2.0     925
0.0     352
3.0     329
4.0      87
5.0      38
Name: client_segment, dtype: int64
0.0    4820
1.0     468
6.0      78
2.0      45
4.0      29
3.0       3
Name: sect_empl, dtype: int64


M    2734
V    2709
Name: gender, dtype: int64

Let's dummify them. I'm going to make the missing a separate category - not sure whether that is better than e.g. a mode imputation?

In [9]:
datafull['client_segment'] = pd.Categorical(datafull['client_segment'])
datafull['sect_empl'] = pd.Categorical(datafull['sect_empl'])
pd.get_dummies(datafull[['client_segment', 'sect_empl']], dummy_na=True).head()

,client_segment_0.0,client_segment_1.0,client_segment_2.0,client_segment_3.0,client_segment_4.0,client_segment_5.0,client_segment_nan,sect_empl_0.0,sect_empl_1.0,sect_empl_2.0,sect_empl_3.0,sect_empl_4.0,sect_empl_6.0,sect_empl_nan
0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,1,0,0,0,0,1,0,0,0,0,0,0


In [10]:
print(datafull.shape)
datafull2 = pd.concat([datafull,pd.get_dummies(datafull[['gender','client_segment', 'sect_empl']], dummy_na=True)], axis=1)
print(datafull2.shape)

(5500, 50)
(5500, 67)


I will stop here, but depending on which types of models you will use, it might be a good idea to remove the original features (or perhaps its better to explicitly keep them?), and perhaps also one dummy? Up to you, but I remove the original features.

In [11]:
datafull2.columns

Index(['income_am', 'profit_last_am', 'profit_am', 'damage_am', 'damage_inc',
       'crd_lim_rec', 'credit_use_ic', 'gluten_ic', 'lactose_ic',
       'insurance_ic', 'spa_ic', 'empl_ic', 'cab_requests', 'married_cd',
       'bar_no', 'sport_ic', 'neighbor_income', 'age', 'marketing_permit',
       'urban_ic', 'dining_ic', 'presidential', 'client_segment', 'sect_empl',
       'prev_stay', 'prev_all_in_stay', 'divorce', 'fam_adult_size',
       'children_no', 'tenure_mts', 'tenure_yrs', 'company_ic', 'claims_no',
       'claims_am', 'nights_booked', 'gender', 'shop_am', 'shop_use',
       'retired', 'gold_status', 'score1_pos', 'score1_neg', 'score2_pos',
       'score2_neg', 'score3_pos', 'score3_neg', 'score4_pos', 'score4_neg',
       'score5_pos', 'score5_neg', 'gender_M', 'gender_V', 'gender_nan',
       'client_segment_0.0', 'client_segment_1.0', 'client_segment_2.0',
       'client_segment_3.0', 'client_segment_4.0', 'client_segment_5.0',
       'client_segment_nan', 'sect_empl_0

In [12]:
print(datafull2.shape)
datafull2.drop(['client_segment', 'sect_empl', 'gender'], axis=1, inplace=True)
print(datafull2.shape)

(5500, 67)
(5500, 64)


I will give one example of a simple engineered feature: profit per night.

In [13]:
datafull2['profitpernight'] = datafull2['profit_am'] / datafull2['nights_booked']

And then there are of course missing values. There is much to be gained or lost by having a good approach here. I keep it simple: I remove features with too many missings (say, more than 30%?), and I remove observations with too many missings (say, more than 30%?) The remainder receives a very simple mean imputation.

In [14]:
Fpi

score1_pos    0.755091
score1_neg    0.736727
score2_pos    0.760000
score2_neg    0.740364
score3_pos    0.746000
score3_neg    0.725091
score4_pos    0.756000
score4_neg    0.736364
score5_pos    0.752000
score5_neg    0.700545
dtype: float64

In [15]:
print(datafull2.shape)
datafull2.dropna(thresh = datafull2.shape[0]*0.3, axis = 1, inplace = True)
print(datafull2.shape)

(5500, 65)
(5500, 55)


It is slightly worrisome that these are all score columns. Perhaps you can find a better way to deal with this?

Just changing the axis allows to remove observations with too many missings. Here none meet our criterion of more than 30% missing.

In [16]:
print(datafull2.shape)
datafull2.dropna(thresh = datafull2.shape[1]*0.3, axis = 0, inplace = True)
print(datafull2.shape)

(5500, 55)
(5500, 55)


We now do a mean impute - which may or may not be optimal, with all those binary features... If you check out the code datafull2.isnull().sum(), you will see that the outcome has no missings, which is of course a good thing, we can just run the imputation through...

In [17]:
print(datafull2.isnull().sum().sum())
datafull2.fillna(datafull2.mean(), inplace=True)
print(datafull2.isnull().sum().sum())

3086
0


Finally, we rescale all the features.

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
datafull3 = pd.DataFrame(scaler.fit_transform(datafull2))
datafull3.columns = datafull2.columns

In the last step, we separate the data sets again.

In [19]:
data = pd.concat([data[['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount']],datafull3[0:5000]], axis=1)
print(data.shape)
score = datafull3[5000:5500]
score.shape

(5000, 58)


(500, 55)

### Model for profit
I'm not going to be too fancy with data splitting: a plain train/test split, and then within train a CV for hyperparameters.

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount'],1), 
                                                    data['outcome_profit'], test_size=0.2, random_state=29949)

Being my lazy self, I just steal the code from the practical, with some minor changes.

In [21]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 50)]
learning_rate = [x for x in np.logspace(start = -3, stop = -0.01, num = 50)]
max_features = ['auto']
max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]
min_samples_split = [2, 5, 10, 30]
min_samples_leaf = [1, 2, 4, 10, 30]
subsample = [0.4, 0.6, 0.8, 1]
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'subsample': subsample}
gbm = GradientBoostingRegressor()
gbm_random = RandomizedSearchCV(estimator = gbm, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=4872, n_jobs = -1)
gbm_random.fit(X_train, y_train)
gbm_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 10.0min finished


{'subsample': 0.4,
 'n_estimators': 577,
 'min_samples_split': 30,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 9,
 'learning_rate': 0.016611473712508432}

Let's fit the final model for profit.

In [22]:
params = gbm_random.best_params_
gbm_profit = GradientBoostingRegressor(**params)
gbm_profit.fit(X_train, y_train)
print('R2: %.3f' % gbm_profit.score(X_train, np.array(y_train).reshape(-1,1)))
print('R2: %.3f' % gbm_profit.score(X_test, np.array(y_test).reshape(-1,1)))

R2: 0.920
R2: 0.752


Now don't forget to score the 500 potential guests!

In [23]:
profit_preds = gbm_profit.predict(score)

Explanations of the model internals can really follow the practical on white-boxing, I won't do it here. Instead, I move straight on to the...

### Model for damage (binary)

In [24]:
data.outcome_damage_inc.value_counts()

0    3723
1    1277
Name: outcome_damage_inc, dtype: int64

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount'],1), 
                                                    data['outcome_damage_inc'], test_size=0.2, random_state=9876)
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 50)]
learning_rate = [x for x in np.logspace(start = -3, stop = -0.01, num = 50)]
max_features = ['auto']
max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]
min_samples_split = [2, 5, 10, 30]
min_samples_leaf = [1, 2, 4, 10, 30]
subsample = [0.4, 0.6, 0.8, 1]
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'subsample': subsample}
gbm = GradientBoostingClassifier()
gbm_random = RandomizedSearchCV(estimator = gbm, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=4872, n_jobs = -1)
gbm_random.fit(X_train, y_train)
gbm_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 10.8min finished


{'subsample': 0.6,
 'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 4,
 'learning_rate': 0.004075717570257835}

In [26]:
params = gbm_random.best_params_
gbm_damagebin = GradientBoostingClassifier(**params)
gbm_damagebin.fit(X_train, y_train)
print('Train accuracy: %.3f' % gbm_damagebin.score(X_train, y_train))
print('Test accuracy: %.3f' % gbm_damagebin.score(X_test, y_test))

Train accuracy: 0.797
Test accuracy: 0.736


In [27]:
damagebin_preds = gbm_damagebin.predict_proba(score)

Note that I did not do any effort at all to think about the issues we discussed in the session about binary data. I know you can do a better job here! Myself, on the other hand, I'm gonna move on.

### Model for damage (amount)
For this, you may want to take care. I already know from the model above who is going to cause damage, so do I want to predict the damage amount for just anyone? No! I want to predict it only for those who actually cause damage, and so I only train it in those that caused damage. 

In [28]:
from sklearn.model_selection import train_test_split
data_dam = data[data.outcome_damage_inc == 1]
data_dam.shape

(1277, 58)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(data_dam.drop(['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount'],1), 
                                                    data_dam['outcome_damage_amount'], test_size=0.2, random_state=29949)

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 50)]
learning_rate = [x for x in np.logspace(start = -3, stop = -0.01, num = 50)]
max_features = ['auto']
max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]
min_samples_split = [2, 5, 10, 30]
min_samples_leaf = [1, 2, 4, 10, 30]
subsample = [0.4, 0.6, 0.8, 1]
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'subsample': subsample}
gbm = GradientBoostingRegressor()
gbm_random = RandomizedSearchCV(estimator = gbm, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=4872, n_jobs = -1)
gbm_random.fit(X_train, y_train)
gbm_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.4min finished


{'subsample': 0.4,
 'n_estimators': 412,
 'min_samples_split': 5,
 'min_samples_leaf': 30,
 'max_features': 'auto',
 'max_depth': 8,
 'learning_rate': 0.009469488710361799}

In [30]:
params = gbm_random.best_params_
gbm_damageam = GradientBoostingRegressor(**params)
gbm_damageam.fit(X_train, y_train)
print('R2: %.3f' % gbm_damageam.score(X_train, np.array(y_train).reshape(-1,1)))
print('R2: %.3f' % gbm_damageam.score(X_test, np.array(y_test).reshape(-1,1)))
damageam_preds = gbm_damageam.predict(score)

R2: 0.371
R2: 0.088


Fieuw, not such a good model! Still, it's up to you to consider whether this is good enough, and what could be better. Me, with my minimal solution, I'm already done! I now works towards a criterion for ranking clients. I am going to subtract the expected damages from the expected profit, but you may want to do something else?

In [31]:
pd.Series(profit_preds - [x[1] for x in damagebin_preds] * damageam_preds)

0      1255.529809
1      2152.721999
2       966.632632
3      1891.705241
4      1350.787637
          ...     
495    1973.134300
496    3108.621444
497    1606.019345
498    1742.387598
499    1242.793813
Length: 500, dtype: float64

In [32]:
exp_profit = pd.Series(profit_preds - [x[1] for x in damagebin_preds] * damageam_preds)
exp_profit.index = score.index
score['exp_profit'] = exp_profit

<ipython-input-32-035d056a5132>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score['exp_profit'] = exp_profit


Finally, I select the best of the potential client list.

In [33]:
score.sort_values('exp_profit', ascending=False).head(10)

,income_am,profit_last_am,profit_am,damage_am,damage_inc,crd_lim_rec,credit_use_ic,gluten_ic,lactose_ic,insurance_ic,spa_ic,empl_ic,cab_requests,married_cd,bar_no,sport_ic,neighbor_income,age,marketing_permit,urban_ic,dining_ic,presidential,prev_stay,prev_all_in_stay,divorce,fam_adult_size,children_no,tenure_mts,tenure_yrs,company_ic,claims_no,claims_am,nights_booked,shop_am,shop_use,retired,gold_status,gender_M,gender_V,gender_nan,client_segment_0.0,client_segment_1.0,client_segment_2.0,client_segment_3.0,client_segment_4.0,client_segment_5.0,client_segment_nan,sect_empl_0.0,sect_empl_1.0,sect_empl_2.0,sect_empl_3.0,sect_empl_4.0,sect_empl_6.0,sect_empl_nan,profitpernight,exp_profit
5081,1.940324,0.088783,4.673283,-0.257888,-0.400697,-0.729723,-0.206795,-0.159701,-0.323565,-0.804458,-0.814459,-0.154389,0.965341,-2.070860,-0.731548,-0.636862,1.254833,1.934358,1.014313,-2.766034,-0.229867,-0.064465,0.352159,-0.585555,-0.340079,-1.205512,-0.463093,-1.689418,-1.703415,-0.136819,-0.306211,-0.066991,-0.746039,-0.303011,-0.427959,2.135877,-0.197347,-0.994199,1.015022,-0.102334,-0.261488,-1.440855,-0.449651,3.964508,-0.126777,-0.08341,-0.102334,0.375605,-0.304967,-0.090826,-0.023361,-0.072806,-0.119941,-0.102334,24.250550,18299.753737
5431,2.224194,-0.195415,5.415988,-0.257888,0.727014,-0.729723,-0.206795,-0.159701,-0.323565,-0.804458,-0.814459,-0.154389,-1.314814,0.482891,-0.329548,-0.636862,-0.881735,0.071430,1.014313,0.365314,-0.229867,-0.064465,0.352159,-0.585555,-0.340079,1.295575,-0.463093,-0.611296,-0.639747,-0.136819,6.798247,-0.066991,-0.666020,-0.303011,-0.427959,-0.473095,5.120278,1.005835,-0.985200,-0.102334,-0.261488,-1.440855,-0.449651,3.964508,-0.126777,-0.08341,-0.102334,-2.662374,3.279045,-0.090826,-0.023361,-0.072806,-0.119941,-0.102334,6.578927,8634.701816
5256,2.648173,-0.226306,5.536532,0.389627,2.982435,0.814962,-0.206795,-0.159701,3.122937,-0.804458,1.234994,-0.154389,-0.989077,0.482891,0.072453,1.586641,-0.924991,0.257723,1.014313,0.365314,-0.229867,15.779189,0.352159,1.725667,-0.340079,1.295575,-0.463093,0.589650,0.587562,-0.136819,3.956464,-0.066991,-0.425961,6.983459,2.376861,-0.473095,5.120278,1.005835,-0.985200,-0.102334,-0.261488,-1.440855,-0.449651,3.964508,-0.126777,-0.08341,-0.102334,0.375605,-0.304967,-0.090826,-0.023361,-0.072806,-0.119941,-0.102334,1.747839,8424.161449
5048,2.553630,-0.050570,3.054425,-0.257888,-0.400697,0.814962,-0.206795,-0.159701,-0.323565,-0.804458,-0.814459,-0.154389,-0.989077,0.482891,0.474454,1.586641,-0.879173,1.561772,1.014313,-2.766034,-0.229867,-0.064465,0.352159,-0.585555,-0.340079,-1.205512,-0.463093,0.753416,0.751203,-0.136819,-0.306211,-0.066991,-0.666020,-0.303011,-0.427959,2.135877,5.120278,1.005835,-0.985200,-0.102334,-0.261488,-1.440855,-0.449651,3.964508,-0.126777,-0.08341,-0.102334,0.375605,-0.304967,-0.090826,-0.023361,-0.072806,-0.119941,-0.102334,3.836675,7686.753629
5078,0.160589,-0.139468,0.656452,-0.257888,1.854724,2.580316,-0.206795,6.327267,3.122937,1.256090,1.234994,-0.154389,0.639605,0.482891,3.489459,1.586641,1.263424,0.071430,-0.996213,0.365314,-0.229867,-0.064465,0.352159,1.725667,-0.340079,1.295575,1.945520,1.394830,1.405769,-0.136819,-0.306211,-0.066991,-0.746039,2.226833,2.376861,-0.473095,5.120278,1.005835,-0.985200,-0.102334,-0.261488,-1.440855,2.223948,-0.252238,-0.126777,-0.08341,-0.102334,0.375605,-0.304967,-0.090826,-0.023361,-0.072806,-0.119941,-0.102334,5.593145,7636.364925
5084,0.553987,0.334195,0.803155,2.625056,7.493277,0.594293,4.886349,-0.159701,3.122937,1.256090,1.234994,-0.154389,0.639605,0.482891,-0.329548,1.586641,0.063112,-0.735839,1.014313,0.365314,-0.229867,-0.064465,0.352159,-0.585555,-0.340079,0.045032,-0.463093,-1.150357,-1.130671,-0.136819,-0.306211,-0.066991,-0.746039,-0.303011,-0.427959,-0.473095,5.120278,-0.994199,1.015022,-0.102334,-0.261488,0.694032,-0.449651,-0.252238,-0.126777,-0.08341,-0.102334,0.375605,-0.304967,-0.090826,-0.023361,-0.072806,-0.119941,-0.102334,6.274551,7429.107987
5253,2.975172,-0.171389,3.02932